In [ ]:

from dotenv import load_dotenv, find_dotenv
from google.cloud.sql.connector import Connector
import os
import pandas as pd
import sqlalchemy

In [ ]:
df = pd.read_parquet("../../Product-Descriptions-and-Ads/data/train-00000-of-00001-36bd1b172faedc74.parquet")

In [ ]:
df[["description"]][:10]

In [ ]:
_ = load_dotenv(find_dotenv())

In [ ]:
def execute_instruction(sql_instruction:str):
    project_id = os.environ["PROJECT_ID"]
    region = os.environ["REGION"]
    instance_name = os.environ["INSTANCE_NAME"]

    INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
    DB_USER = os.environ["DB_USER"]
    DB_PASS = os.environ["DB_PASS"]
    DB_NAME = os.environ["DB_NAME"]

    connector = Connector()

    # function to return the database connection object
    def getconn():
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pg8000",
            user=DB_USER,
            password=DB_PASS,
            db=DB_NAME
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
    )

    # connect to connection pool
    with pool.connect() as db_conn:

    # query and fetch ratings table
    #   results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()
#         sql_instruction = '''SELECT datname FROM pg_database
# WHERE datistemplate = false;'''
        results = db_conn.execute(sqlalchemy.text(sql_instruction))

    # show results
        # for row in results:
        #     print(row)

    connector.close()
    return results

In [ ]:
inst = """CREATE EXTENSION vector;"""
val = execute_instruction(inst)
val

In [ ]:
# Save the Pandas dataframe in a PostgreSQL table.
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector

async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}"
        )

        # Create the `products` table.	 
        await conn.execute("""CREATE TABLE products(
                                product_id VARCHAR(1024) PRIMARY KEY,
                                product_name TEXT,
                                description TEXT,
                                list_price NUMERIC)""")

        # Copy the dataframe to the `products` table.
        tuples = list(df.itertuples(index=False))
        await conn.copy_records_to_table('products', records=tuples, columns=list(df), timeout=10)
        await conn.close()


# Run the SQL commands now.
await main() # type: ignore

In [ ]:
def get_connection():
    project_id = os.environ["PROJECT_ID"]
    region = os.environ["REGION"]
    instance_name = os.environ["INSTANCE_NAME"]

    INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
    DB_USER = os.environ["DB_USER"]
    DB_PASS = os.environ["DB_PASS"]
    DB_NAME = os.environ["DB_NAME"]

    connector = Connector()

    # function to return the database connection object
    def getconn():
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pg8000",
            user=DB_USER,
            password=DB_PASS,
            db=DB_NAME
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
    )

    # connect to connection pool
    #with pool.connect() as db_conn:

    # query and fetch ratings table
    #   results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()
#         sql_instruction = '''SELECT datname FROM pg_database
# WHERE datistemplate = false;'''
        #results = db_conn.execute(sqlalchemy.text(sql_instruction))

    # show results
        # for row in results:
        #     print(row)

    #connector.close()
    return pool

In [ ]:
pool_ = get_connection()

In [ ]:
inst = """SELECT datname FROM pg_database"""

In [ ]:
with pool_.connect() as db_conn:
    results = db_conn.execute(sqlalchemy.text(inst))
print(results.fetchall())